# Exploratory Data Analysis

### Telco Churn

In [ ]:
# Imports (keep in a single top cell)
import os
import io
import json
import subprocess
import shlex
from pathlib import Path

from datetime import datetime

import yaml

import boto3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 100)
sns.set_theme()

In [ ]:
# Step 3.b — Load project environment variables (with granular prefixes)

REQUIRED_BASE = ["AWS_REGION", "ACCOUNT_ID", "LAB_PREFIX", "BUCKET", "S3_DATA", "S3_ARTIFACTS"]
OPTIONAL_FINE = ["S3_CODE", "S3_DATA_RAW", "S3_DATA_PROCESSED", "S3_ART_PREPROCESS"]

def _source_env_file(env_file="~/mlops-env.sh", wanted=(REQUIRED_BASE + OPTIONAL_FINE)) -> dict:
    path = Path(env_file).expanduser()
    if not path.exists():
        return {}
    cmd = f"bash -lc 'set -a; source {shlex.quote(str(path))} >/dev/null 2>&1; env'"
    out = subprocess.check_output(cmd, shell=True, text=True)
    seen = {}
    for line in out.splitlines():
        if "=" not in line:
            continue
        k, v = line.split("=", 1)
        if k in wanted:
            seen[k] = v
    return seen

# Import what we can from current kernel env, then from ~/mlops-env.sh if needed
missing = [k for k in REQUIRED_BASE if not os.environ.get(k)]
if missing:
    os.environ.update(_source_env_file())

# Error if base vars still missing
missing = [k for k in REQUIRED_BASE if not os.environ.get(k)]
if missing:
    raise OSError(
        "Missing environment variables: "
        + ", ".join(missing)
        + "\nFix in VS Code Terminal: source ~/mlops-env.sh, then restart the kernel and re-run."
    )

# Bind coarse vars (from earlier labs)
REGION       = os.environ["AWS_REGION"]
ACCOUNT_ID   = os.environ["ACCOUNT_ID"]
LAB_PREFIX   = os.environ["LAB_PREFIX"]
BUCKET       = os.environ["BUCKET"]
S3_DATA      = os.environ["S3_DATA"]          # s3://.../data
S3_ARTIFACTS = os.environ["S3_ARTIFACTS"]     # s3://.../artifacts

# Try to read granular vars; if absent, derive them from the coarse ones (backward compatible)
S3_CODE            = os.environ.get("S3_CODE") or f"s3://{BUCKET}/{LAB_PREFIX}/code"
S3_DATA_RAW        = os.environ.get("S3_DATA_RAW") or f"{S3_DATA.rstrip('/')}/raw"
S3_DATA_PROCESSED  = os.environ.get("S3_DATA_PROCESSED") or f"{S3_DATA.rstrip('/')}/processed"
S3_ART_PREPROCESS  = os.environ.get("S3_ART_PREPROCESS") or f"{S3_ARTIFACTS.rstrip('/')}/preprocess"

print(json.dumps({
    "REGION": REGION, "ACCOUNT_ID": ACCOUNT_ID, "LAB_PREFIX": LAB_PREFIX, "BUCKET": BUCKET,
    "S3_CODE": S3_CODE, "S3_DATA_RAW": S3_DATA_RAW, "S3_DATA_PROCESSED": S3_DATA_PROCESSED,
    "S3_ART_PREPROCESS": S3_ART_PREPROCESS
}, indent=2))



In [ ]:
# One session & client for the whole notebook (faster; cleaner; testable)
session = boto3.Session(region_name=REGION)
s3 = session.client("s3")

# Sanity check: who am I? (no secrets printed)
sts = session.client("sts")
who = sts.get_caller_identity()
print("Caller identity:", json.dumps(who, indent=2))

In [ ]:
# Where the Telco CSVs live by convention in this course:
#   s3://<bucket>/data/raw/telco/
# You can override via TELCO_PREFIX in your shell if you used a different folder.
TELCO_PREFIX = os.environ.get("TELCO_PREFIX") or f"{S3_DATA_RAW.rstrip('/')}/telco/"
print("TELCO_PREFIX:", TELCO_PREFIX)

In [ ]:
def split_s3_uri(s3_uri: str) -> tuple[str, str]:
    """Turn 's3://bucket/prefix' -> ('bucket','prefix')."""
    if not s3_uri.startswith("s3://"):
        raise ValueError(f"Not an s3 URI: {s3_uri}")
    no_scheme = s3_uri[len("s3://") :]
    parts = no_scheme.split("/", 1)
    bucket = parts[0]
    prefix = "" if len(parts) == 1 else parts[1]
    return bucket, prefix

def list_csv_objects(s3_client, bucket: str, prefix: str) -> list[str]:
    """List .csv keys under a prefix (handles pagination)."""
    keys, token = [], None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix}
        if token:
            kwargs["ContinuationToken"] = token
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            key = obj["Key"]
            if key.lower().endswith(".csv"):
                keys.append(key)
        if resp.get("IsTruncated"):
            token = resp["NextContinuationToken"]
        else:
            break
    return keys


In [ ]:
bucket, telco_prefix_key = split_s3_uri(TELCO_PREFIX)
csv_keys = list_csv_objects(s3, bucket=bucket, prefix=telco_prefix_key)

print(f"Found {len(csv_keys)} CSV file(s) under {TELCO_PREFIX}")
for k in csv_keys[:10]:
    print(" -", k)

if not csv_keys:
    raise FileNotFoundError(
        f"No CSVs under {TELCO_PREFIX}\n"
        "Upload your Telco CSV to this folder and re-run this cell."
    )

def read_csv_from_s3(s3_client, bucket: str, key: str, **pd_kwargs) -> pd.DataFrame:
    """
    Stream a single CSV from S3 into a pandas DataFrame.
    Tries utf-8 first, then falls back to latin-1 for quirky files.
    """
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    body = obj["Body"].read()  # bytes
    try:
        return pd.read_csv(io.BytesIO(body), encoding="utf-8", **pd_kwargs)
    except UnicodeDecodeError:
        return pd.read_csv(io.BytesIO(body), encoding="latin-1", **pd_kwargs)

TARGET = "Churn"  # keep consistent across labs

if len(csv_keys) == 1:
    print("Loading single file…")
    df = read_csv_from_s3(s3, bucket, csv_keys[0], header=0)
else:
    print("Multiple files detected; loading and concatenating…")
    frames = []
    for i, key in enumerate(csv_keys, start=1):
        print(f"  [{i}/{len(csv_keys)}] {key}")
        part = read_csv_from_s3(s3, bucket, key, header=0)
        frames.append(part)
    df = pd.concat(frames, axis=0, ignore_index=True)

print(f"Loaded shape: {df.shape}")
display(df.head(3))

In [ ]:
print("Columns:", list(df.columns))
print("\nDtypes:\n", df.dtypes)

if TARGET in df.columns:
    print("\nTarget distribution (raw):")
    print(df[TARGET].value_counts(dropna=False))
else:
    raise KeyError(f"Expected target column '{TARGET}' not found. "
                   "Check your file(s) and header row.")

In [ ]:
# Keep a reference to the original dataframe from Step 3
df_raw = df.copy()

# Quick overview
print("Rows, Cols:", df_raw.shape)
display(df_raw.sample(min(5, len(df_raw)), random_state=42))
display(df_raw.dtypes)

In [ ]:
# Work on a clean copy
df = df_raw.copy()

# 1) Coerce 'TotalCharges' to numeric (bad parses become NaN)
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# 2) Drop identifier columns if present
for col in ["customerID", "CustomerID", "customer_id"]:
    if col in df.columns:
        df = df.drop(columns=[col])

# 3) (Optional) Trim surrounding whitespace in string columns
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip()

# 4) Make sure the target exists and is string/categorical
TARGET = "Churn"
if TARGET not in df.columns:
    raise KeyError(f"Expected target column '{TARGET}' not found. Check your header and file(s).")

# Cast target to category (helps some downstream tools)
df[TARGET] = df[TARGET].astype("category")

# Show the result
df.info()

In [ ]:
# Nulls per column (we'll impute later in the processing pipeline)
null_summary = df.isna().sum().sort_values(ascending=False)
display(null_summary[null_summary > 0].head(15))

# Target distribution (class imbalance is typical: more 'No' than 'Yes')
print("\nTarget distribution:")
print(df[TARGET].value_counts(dropna=False))

In [ ]:
assert "Churn" in df.columns, "Expected 'Churn' column"
vc = df["Churn"].value_counts(dropna=False)
display(vc.to_frame("count").assign(share=lambda t: (t["count"]/t["count"].sum()).round(3)))

sns.barplot(x=vc.index, y=vc.values)
plt.title("Target distribution: Churn"); plt.xlabel("Churn"); plt.ylabel("Count"); plt.show()

In [ ]:
null_counts = df.isna().sum().sort_values(ascending=False)
miss = (
    null_counts[null_counts > 0]
    .to_frame(name="null_count")
    .assign(null_rate=lambda t: (t["null_count"] / len(df)).round(4))
)

if miss.empty:
    print("No missing values detected.")
else:
    display(miss)

    # Bar chart (clearer than a full heatmap when few cols have NaNs)
    ax = miss.sort_values("null_count").plot(
        kind="barh", y="null_count", legend=False, figsize=(8, 4)
    )
    plt.title("Columns with missing values")
    plt.xlabel("Rows with NaN")
    plt.tight_layout()
    plt.show()

In [ ]:
if "TotalCharges" in df.columns:
    n_nan = int(df["TotalCharges"].isna().sum())
    rate = n_nan / len(df)
    print(f"TotalCharges NaN rows: {n_nan} ({rate:.2%})")

    if n_nan:
        cols_to_show = [c for c in ["tenure", "MonthlyCharges", "TotalCharges"] if c in df.columns]
        display(df.loc[df["TotalCharges"].isna(), cols_to_show].head(10))

        if "tenure" in df.columns:
            vc = df.loc[df["TotalCharges"].isna(), "tenure"].value_counts().sort_index()
            print("tenure values among TotalCharges==NaN:")
            display(vc.to_frame("rows"))

In [ ]:
if not miss.empty and miss["null_count"].sum() > 0:
    plt.figure(figsize=(8, 4))
    # Show at most 1000 rows to keep it readable
    subset = df.isna().iloc[: min(1000, len(df))]
    sns.heatmap(subset, cbar=False)
    plt.title("Missingness map (first 1000 rows)")
    plt.xlabel("Columns"); plt.ylabel("Rows (subset)")
    plt.show()

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c != "Churn"]
print("Numeric columns:", num_cols)

df[num_cols].hist(figsize=(12, 8), bins=30)
plt.suptitle("Numeric distributions", y=1.02); plt.show()

def iqr_outliers(s: pd.Series) -> int:
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    return int(((s < lo) | (s > hi)).sum())

outlier_summary = pd.Series({c: iqr_outliers(df[c].dropna()) for c in num_cols}).sort_values(ascending=False)
display(outlier_summary.to_frame("iqr_outlier_count"))

In [ ]:
cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
cat_cols = [c for c in cat_cols if c != "Churn"]

def churn_rate_by(cat_col, top=12):
    t = (df.groupby(cat_col)["Churn"]
           .value_counts(normalize=True)
           .rename("share")
           .mul(100)
           .reset_index())
    return t[t["Churn"] == "Yes"].sort_values("share", ascending=False).head(top)

for col in [c for c in ["Contract", "PaymentMethod", "InternetService"] if c in cat_cols]:
    display(churn_rate_by(col))

if "Contract" in cat_cols:
    rates = churn_rate_by("Contract", top=10)
    sns.barplot(data=rates, x="Contract", y="share")
    plt.title("Churn rate by Contract (%)")
    plt.ylabel("Churn Yes (%)"); plt.xlabel("Contract"); plt.show()

In [ ]:
from sklearn.metrics import mutual_info_score

df_corr = df.copy()
df_corr["ChurnBin"] = (df_corr["Churn"].map({"Yes": 1, "No": 0})).astype("float")

num_corr = df_corr[num_cols + ["ChurnBin"]].corr()["ChurnBin"].drop("ChurnBin").sort_values(ascending=False)
display(num_corr.to_frame("pearson_corr_with_churn"))

plt.figure(figsize=(6, 8))
sns.barplot(x=num_corr.values, y=num_corr.index)
plt.title("Numeric features: correlation with Churn (Yes=1)")
plt.xlabel("Pearson r"); plt.ylabel("Feature"); plt.show()

def mi_cat(col):
    return mutual_info_score(df_corr[col].astype(str), df_corr["Churn"])
mi = pd.Series({c: mi_cat(c) for c in cat_cols}).sort_values(ascending=False)
display(mi.to_frame("mutual_info_with_churn"))

In [ ]:
# Step 11 — Save concise EDA summary to $S3_ART_PREPROCESS/eda/
summary = {
    "rows": int(len(df)),
    "cols": int(df.shape[1]),
    "target": "Churn",
    "target_counts": df["Churn"].value_counts(dropna=False).to_dict(),
    "null_columns": {k: int(v) for k, v in df.isna().sum().items() if v > 0},
    "top_numeric_corr": (num_corr.head(5).round(3).to_dict() if "num_corr" in globals() else {}),
    "top_categorical_mi": (mi.head(5).round(3).to_dict() if "mi" in globals() else {}),
    "generated_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",
}
print(json.dumps(summary, indent=2))

stamp = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
local_name = f"eda_notebook_summary_{stamp}.json"
local_path = Path.cwd() / local_name
local_path.write_text(json.dumps(summary, indent=2))
print(f"Wrote: {local_path}")

# Upload to $S3_ART_PREPROCESS/eda/
eda_bucket, eda_prefix = split_s3_uri(f"{S3_ART_PREPROCESS.rstrip('/')}/eda/")
eda_key = f"{eda_prefix.rstrip('/')}/{local_name}"
s3.upload_file(str(local_path), eda_bucket, eda_key)
print("Uploaded to:", f"s3://{eda_bucket}/{eda_key}")

In [ ]:
# Step 12 — Write plan.yaml (decisions derived from EDA) to $S3_ART_PREPROCESS/

plan = {
    "target": "Churn",
    "splits": {"test_size": 0.20, "val_size": 0.10, "random_state": 42, "stratify": True},
    "numeric_imputer": "median",              # robust; matches TotalCharges insights
    "categorical_imputer": "most_frequent",   # simple, effective for Telco
    "encoder": "onehot_ignore_unknown",       # avoids crashes at inference
    "scale_numeric": True,                    # StandardScaler on numeric slice
    "label_mapping": {"No": 0, "Yes": 1},
    "class_weight": "balanced",               # handle imbalance
    # Optional context for reviewers:
    "notes": {
        "why_numeric_median": "skew & outliers; median is robust",
        "why_ignore_unknown": "stability with new categories at inference",
        "eda_artifact_hint": "see $S3_ART_PREPROCESS/eda/ for report json"
    },
}

# Save alongside the notebook
plan_path = Path.cwd() / "plan.yaml"
plan_path.write_text(yaml.safe_dump(plan, sort_keys=False))
print("Local plan.yaml written:", plan_path)

# Upload to $S3_ART_PREPROCESS/plan.yaml
pbucket, pprefix = split_s3_uri(S3_ART_PREPROCESS)
plan_key = f"{pprefix.rstrip('/')}/plan.yaml"
s3.upload_file(str(plan_path), pbucket, plan_key)
print("Uploaded plan to:", f"s3://{pbucket}/{plan_key}")

In [ ]:
# Quick existence checks
resp = s3.list_objects_v2(Bucket=pbucket, Prefix=pprefix.rstrip('/') + "/")
for obj in resp.get("Contents", []):
    if obj["Key"].endswith("plan.yaml") or "eda_notebook_summary" in obj["Key"]:
        print("✓", obj["Key"])